Notebook to run the ABM multiple time and collect its output to validate it.

## Content
* Real data loading
* Model runs and output collection

In [1]:
# sbp_payments.xlsx for real values of payments; sbp_payments_all0.xlsx for always null payments (counterfactual).
sbp_payments_path = "./data/sbp_payments.xlsx"

# The ML models used are the ones in the he ones in the folder .abm.ml_model

# TO SPECIFY
specific_model = 'all_adoption'

In [2]:
cd ..

C:\Users\giaco\sbp-adoption-abm-operational\abm


In [3]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import geopandas as gpd
import os
import csv

from abm.model import SBPAdoption

from abm.custom_transformers import (
    TransformCensusFeatures,
    TransformClimateFeatures,
    TransformSoilFeatures,
    )

In [4]:
%matplotlib inline

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [5]:
# Line necessary to import agents in model, since the cwd of the notebook server is always the base path where you started the 
# server 
import sys
sys.path.append(".\abm")

In [6]:
# How to display floats
pd.options.display.float_format = '{:.6f}'.format
#np.set_printoptions(precision=6)
np.set_printoptions(formatter={'float': lambda x: "{0:0.6f}".format(x)})

In [7]:
# Where to save the figures
PROJECT_ROOT_DIR = "model_validation"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "output")
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=False, fig_extension="jpeg", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Real data loading

In [8]:
path_to_adoption_port_real = "./model_validation/data/SBP yearly adoption - Portugal.csv"
yearly_adoption_portugal_real = pd.read_csv(path_to_adoption_port_real, index_col=0)
yearly_adoption_portugal_real.drop(1995, inplace=True)

In [9]:
path_to_adoption_munic_real = "./model_validation/data/SBP yearly adoption - Municipalities.csv"
munic_yearly_adoption_real = pd.read_csv(path_to_adoption_munic_real, index_col=['Municipality', 'Year'])

# Model runs and output collection

In [10]:
n_runs = 50

In [11]:
start_year = 1996 # Included in the simulation
stop_year = 2021 # Not included in the simulation

### Set dataframe and function to collect municipalities yearly adoption

In [12]:
municipalities_data = gpd.read_file("./data/municipalities_shp/shapefile_for_munic_abm.shp")
municipalities_data.rename(columns={'Municipali': 'Municipality'},
                           inplace=True)
municipalities = municipalities_data['Municipality'].values.tolist()

years = np.arange(start_year, stop_year)
munic_yearly_adoption = pd.MultiIndex.from_product([municipalities, years])
munic_yearly_adoption = munic_yearly_adoption.to_frame()
munic_yearly_adoption.drop(munic_yearly_adoption.columns, axis=1, inplace=True)
munic_yearly_adoption.index.rename(['Municipality', 'Year'], inplace=True)

In [13]:
def collect_agent_level_yearly_adoption(abm_model, n_run, df_adoption):
    """
    Function to collect data at the individual agent level
    
    Returns
    -------
    municipalities_adoption : pd DataFrame
        Multiindex DataFrame reporting for each muninicipality and for each year the ratio of SBP adopted
    """
    municipality_agents = abm_model.schedule.agents
    col_name = 'Run ' + str(n_run+1)
    df_adoption.loc[:, col_name] = None
    
    # Create a dictionary mapping to each municipality name the Series of its adoption in each year
    yearly_adoptions = [munic.yearly_adoption for munic in municipality_agents]
    map_dict = dict(zip(municipalities, yearly_adoptions))
    for k, v in map_dict.items():
        map_dict[k] = pd.Series(v)
        map_dict[k].name = col_name # Necessary to use .udpdate() method

    # For each municipality, updates the values of adoption    
    for munic in municipalities:
        df = df_adoption.loc[munic]
        df.update(map_dict[munic])
        df_adoption.loc[munic] = df.values

### Set dataframes to collect yearly adoption and cumulative yearly adoption in Portugal

In [14]:
yearly_adoption_portugal_pred = pd.DataFrame(index=years)
yearly_adoption_portugal_pred.index.rename('Year', inplace=True)

cumulative_adoption_portugal_pred = pd.DataFrame(index=years)
cumulative_adoption_portugal_pred.index.rename('Year', inplace=True)

In [15]:
def collect_data_run(model, n_run):
    # Individual adoption
    collect_agent_level_yearly_adoption(model, n_run, munic_yearly_adoption)
    
    # Aggregate adoption
    aggr_adoption_out = model.datacollector.get_model_vars_dataframe()
    aggr_adoption_out.set_index('Year', inplace=True)
    col_name = 'Run ' + str(n_run+1)
    yearly_adoption_portugal_pred[col_name] = aggr_adoption_out['Yearly area of SBP sown [ha/y]']
    cumulative_adoption_portugal_pred[col_name] = aggr_adoption_out['Cumulative area of SBP sown [ha]']

In [16]:
%%time
for i in range(n_runs):
    model = SBPAdoption(initial_year=start_year, seed=i, sbp_payments_path=sbp_payments_path)
    for y in range(start_year, stop_year):
        model.step()
        print("Run n° " + str(i+1) + ": year " + str(y) + " step completed.")
    collect_data_run(model, i)

Run n° 1: year 1996 step completed.
Run n° 1: year 1997 step completed.
Run n° 1: year 1998 step completed.
Run n° 1: year 1999 step completed.
Run n° 1: year 2000 step completed.
Run n° 1: year 2001 step completed.
Run n° 1: year 2002 step completed.
Run n° 1: year 2003 step completed.
Run n° 1: year 2004 step completed.
Run n° 1: year 2005 step completed.
Run n° 1: year 2006 step completed.
Run n° 1: year 2007 step completed.
Run n° 1: year 2008 step completed.
Run n° 1: year 2009 step completed.
Run n° 1: year 2010 step completed.
Run n° 1: year 2011 step completed.
Run n° 1: year 2012 step completed.
Run n° 1: year 2013 step completed.
Run n° 1: year 2014 step completed.
Run n° 1: year 2015 step completed.
Run n° 1: year 2016 step completed.
Run n° 1: year 2017 step completed.
Run n° 1: year 2018 step completed.
Run n° 1: year 2019 step completed.
Run n° 1: year 2020 step completed.
Run n° 2: year 1996 step completed.
Run n° 2: year 1997 step completed.
Run n° 2: year 1998 step com

C:\Users\giaco\anaconda3\envs\mesa_geo_ml\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


Run n° 3: year 1996 step completed.
Run n° 3: year 1997 step completed.
Run n° 3: year 1998 step completed.
Run n° 3: year 1999 step completed.
Run n° 3: year 2000 step completed.
Run n° 3: year 2001 step completed.
Run n° 3: year 2002 step completed.
Run n° 3: year 2003 step completed.
Run n° 3: year 2004 step completed.
Run n° 3: year 2005 step completed.
Run n° 3: year 2006 step completed.
Run n° 3: year 2007 step completed.
Run n° 3: year 2008 step completed.
Run n° 3: year 2009 step completed.
Run n° 3: year 2010 step completed.
Run n° 3: year 2011 step completed.
Run n° 3: year 2012 step completed.
Run n° 3: year 2013 step completed.
Run n° 3: year 2014 step completed.
Run n° 3: year 2015 step completed.
Run n° 3: year 2016 step completed.
Run n° 3: year 2017 step completed.
Run n° 3: year 2018 step completed.
Run n° 3: year 2019 step completed.
Run n° 3: year 2020 step completed.


C:\Users\giaco\anaconda3\envs\mesa_geo_ml\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


Run n° 4: year 1996 step completed.
Run n° 4: year 1997 step completed.
Run n° 4: year 1998 step completed.
Run n° 4: year 1999 step completed.
Run n° 4: year 2000 step completed.
Run n° 4: year 2001 step completed.
Run n° 4: year 2002 step completed.
Run n° 4: year 2003 step completed.
Run n° 4: year 2004 step completed.
Run n° 4: year 2005 step completed.
Run n° 4: year 2006 step completed.
Run n° 4: year 2007 step completed.
Run n° 4: year 2008 step completed.
Run n° 4: year 2009 step completed.
Run n° 4: year 2010 step completed.
Run n° 4: year 2011 step completed.
Run n° 4: year 2012 step completed.
Run n° 4: year 2013 step completed.
Run n° 4: year 2014 step completed.
Run n° 4: year 2015 step completed.
Run n° 4: year 2016 step completed.
Run n° 4: year 2017 step completed.
Run n° 4: year 2018 step completed.
Run n° 4: year 2019 step completed.
Run n° 4: year 2020 step completed.


...

Run n° 49: year 1996 step completed.
Run n° 49: year 1997 step completed.
Run n° 49: year 1998 step completed.
Run n° 49: year 1999 step completed.
Run n° 49: year 2000 step completed.
Run n° 49: year 2001 step completed.
Run n° 49: year 2002 step completed.
Run n° 49: year 2003 step completed.
Run n° 49: year 2004 step completed.
Run n° 49: year 2005 step completed.
Run n° 49: year 2006 step completed.
Run n° 49: year 2007 step completed.
Run n° 49: year 2008 step completed.
Run n° 49: year 2009 step completed.
Run n° 49: year 2010 step completed.
Run n° 49: year 2011 step completed.
Run n° 49: year 2012 step completed.
Run n° 49: year 2013 step completed.
Run n° 49: year 2014 step completed.
Run n° 49: year 2015 step completed.
Run n° 49: year 2016 step completed.
Run n° 49: year 2017 step completed.
Run n° 49: year 2018 step completed.
Run n° 49: year 2019 step completed.
Run n° 49: year 2020 step completed.


C:\Users\giaco\anaconda3\envs\mesa_geo_ml\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


Run n° 50: year 1996 step completed.
Run n° 50: year 1997 step completed.
Run n° 50: year 1998 step completed.
Run n° 50: year 1999 step completed.
Run n° 50: year 2000 step completed.
Run n° 50: year 2001 step completed.
Run n° 50: year 2002 step completed.
Run n° 50: year 2003 step completed.
Run n° 50: year 2004 step completed.
Run n° 50: year 2005 step completed.
Run n° 50: year 2006 step completed.
Run n° 50: year 2007 step completed.
Run n° 50: year 2008 step completed.
Run n° 50: year 2009 step completed.
Run n° 50: year 2010 step completed.
Run n° 50: year 2011 step completed.
Run n° 50: year 2012 step completed.
Run n° 50: year 2013 step completed.
Run n° 50: year 2014 step completed.
Run n° 50: year 2015 step completed.
Run n° 50: year 2016 step completed.
Run n° 50: year 2017 step completed.
Run n° 50: year 2018 step completed.
Run n° 50: year 2019 step completed.
Run n° 50: year 2020 step completed.


C:\Users\giaco\anaconda3\envs\mesa_geo_ml\lib\site-packages\pandas\core\frame.py:6393: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


Wall time: 51min 15s


In [24]:
munic_yearly_adoption.head(10)

Run 1    Run 2    Run 3    Run 4    Run 5    Run 6  \
Municipality Year                                                         
Évora        1996 0.000657 0.000657 0.000657 0.000657 0.000657 0.000657   
             1997 0.000834 0.000833 0.000834 0.000833 0.000834 0.000834   
             1998 0.001072 0.001070 0.001073 0.001071 0.001073 0.001071   
             1999 0.001382 0.001380 0.001382 0.001381 0.001381 0.001381   
             2000 0.001767 0.001767 0.001766 0.001766 0.001767 0.001766   
             2001 0.002230 0.002230 0.002224 0.002224 0.002228 0.002224   
             2002 0.002732 0.002737 0.002717 0.002721 0.002737 0.002720   
             2003 0.003229 0.003239 0.003212 0.003209 0.003241 0.003207   
             2004 0.003656 0.003672 0.003626 0.003642 0.003673 0.003626   
             2005 0.003929 0.003951 0.003880 0.003908 0.003957 0.003888   

                     Run 7    Run 8    Run 9   Run 10  ...   Run 41   Run 42  \
Municipality Year                                      ...                     
Évora        1996 0.000657 0.000657 0.000657 0.000657  ... 0.000657 0.000657   
             1997 0.000833 0.000834 0.000832 0.000830  ... 0.000835 0.000834   
             1998 0.001072 0.001073 0.001068 0.001068  ... 0.001073 0.001072   
             1999 0.001381 0.001381 0.001378 0.001377  ... 0.001383 0.001381   
             2000 0.001766 0.001767 0.001766 0.001765  ... 0.001767 0.001767   
             2001 0.002224 0.002228 0.002232 0.002232  ... 0.002223 0.002229   
             2002 0.002720 0.002729 0.002743 0.002746  ... 0.002717 0.002732   
             2003 0.003206 0.003227 0.003257 0.003266  ... 0.003199 0.003233   
             2004 0.003614 0.003654 0.003706 0.003722  ... 0.003602 0.003660   
             2005 0.003863 0.003926 0.004008 0.004031  ... 0.003862 0.003930   

                    Run 43   Run 44   Run 45   Run 46   Run 47   Run 48  \
Municipality Year                                                         
Évora        1996 0.000657 0.000657 0.000657 0.000657 0.000657 0.000657   
             1997 0.000833 0.000831 0.000836 0.000832 0.000835 0.000834   
             1998 0.001072 0.001069 0.001074 0.001071 0.001073 0.001072   
             1999 0.001381 0.001379 0.001383 0.001380 0.001383 0.001382   
             2000 0.001766 0.001766 0.001767 0.001765 0.001767 0.001766   
             2001 0.002223 0.002232 0.002224 0.002224 0.002222 0.002223   
             2002 0.002718 0.002742 0.002723 0.002722 0.002712 0.002725   
             2003 0.003205 0.003253 0.003225 0.003211 0.003188 0.003217   
             2004 0.003611 0.003702 0.003649 0.003626 0.003581 0.003632   
             2005 0.003854 0.004004 0.003918 0.003887 0.003818 0.003893   

                    Run 49   Run 50  
Municipality Year                    
Évora        1996 0.000657 0.000657  
             1997 0.000835 0.000832  
             1998 0.001073 0.001071  
             1999 0.001382 0.001380  
             2000 0.001767 0.001766  
             2001 0.002225 0.002229  
             2002 0.002720 0.002744  
             2003 0.003209 0.003256  
             2004 0.003620 0.003700  
             2005 0.003875 0.003991  

[10 rows x 50 columns]

## Save model output

In [25]:
os.mkdir("model_validation/output/output")
munic_yearly_adoption.to_csv("model_validation/output/output/municipalities_yearly_adoption.csv")
yearly_adoption_portugal_pred.to_csv("model_validation/output/output/portugal_yearly_adoption.csv")
cumulative_adoption_portugal_pred.to_csv("model_validation/output/output/portugal_cumulative_adoption.csv")

FileExistsError: [WinError 183] Impossibile creare un file, se il file esiste già: 'model_validation/output/output'

## Create output to plot map

In [ ]:
import geopandas as gpd

In [26]:
def collect_agent_level_for_map(abm_model, year):
    """
    Function to collect cumulative adoption in a certain year (in ha and in fraction over pastures area)
    and the shape of the municialities in order to plot the map.
    
    Returns
    -------
    munic_cumul_adoption_and_shape : pd DataFrame
        DataFrame reporting for each muninicipality the total cumulative adoption in ha and its shape
    """
    municipality_agents = abm_model.schedule.agents
    #municipalities = [munic.Municipality for munic in municipality_agents]
    
    #munic_cumul_adoption_and_shape = pd.DataFrame(index=municipalities)
    munic_cumul_adoption_and_shape = gpd.GeoDataFrame()
    munic_cumul_adoption_and_shape['Total cumulative adoption [ha]'] = None
    munic_cumul_adoption_and_shape['Total cumulative adoption [fraction]'] = None
    munic_cumul_adoption_and_shape['geometry'] = None
    
    for munic in municipality_agents:
        yearly_adoption_until_year_ha = [v for k, v in munic.yearly_adoption_ha.items()
                                        if k <= year]
        munic_tot_cumul_adoption_year_ha = sum(yearly_adoption_until_year_ha)
        
        yearly_adoption_until_year = [v for k, v in munic.yearly_adoption.items()
                                     if k <= year]
        munic_tot_cumul_adoption_year = sum(yearly_adoption_until_year)
        
        munic_cumul_adoption_and_shape.loc[munic.Municipality] = (
            munic_tot_cumul_adoption_year_ha, 
            munic_tot_cumul_adoption_year,
            munic.shape
        )
    
    return munic_cumul_adoption_and_shape

In [27]:
year_for_map = 2012

In [28]:
pred_for_map = collect_agent_level_for_map(model, year_for_map)

In [23]:
pred_for_map.to_csv("model_validation/output/output/adoption_pred_for_map.csv")